In [4]:
pip install librosa

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from os import *
import librosa
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.layers.experimental import preprocessing
from keras.utils import to_categorical

Виртуальный привод для гугл диска

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
path = '/content/gdrive/My Drive/ru_data/icemarch'
soundlist = sorted(listdir(path))
for file in soundlist:
  print(file)

icemarch_0001.wav
icemarch_0002.wav
icemarch_0003.wav
icemarch_0004.wav
icemarch_0005.wav
icemarch_0006.wav
icemarch_0007.wav
icemarch_0008.wav
icemarch_0009.wav
icemarch_0010.wav
icemarch_0011.wav
icemarch_0012.wav
icemarch_0013.wav
icemarch_0014.wav
icemarch_0015.wav
icemarch_0016.wav
icemarch_0017.wav
icemarch_0018.wav
icemarch_0019.wav
icemarch_0020.wav
icemarch_0021.wav
icemarch_0022.wav
icemarch_0023.wav
icemarch_0024.wav
icemarch_0025.wav
icemarch_0026.wav
icemarch_0027.wav
icemarch_0028.wav
icemarch_0029.wav
icemarch_0030.wav
icemarch_0031.wav
icemarch_0032.wav
icemarch_0033.wav
icemarch_0034.wav
icemarch_0035.wav
icemarch_0036.wav
icemarch_0037.wav
icemarch_0038.wav
icemarch_0039.wav
icemarch_0040.wav
icemarch_0041.wav
icemarch_0042.wav
icemarch_0043.wav
icemarch_0044.wav
icemarch_0045.wav
icemarch_0046.wav
icemarch_0047.wav
icemarch_0048.wav
icemarch_0049.wav
icemarch_0050.wav
icemarch_0051.wav
icemarch_0052.wav
icemarch_0053.wav
icemarch_0054.wav
icemarch_0055.wav
icemarch_0

In [7]:
vocabulary = { 'а': 1,
               'б': 2,
               'в': 3,
               'г': 4,
               'д': 5,
               'е': 6,
               'ё': 7,
               'ж': 8,
               'з': 9,
               'и': 10,
               'й': 11,
               'к': 12,
               'л': 13,
               'м': 14,
               'н': 15,
               'о': 16,
               'п': 17,
               'р': 18,
               'с': 19,
               'т': 20,
               'у': 21,
               'ф': 22,
               'х': 23,
               'ц': 24,
               'ч': 25,
               'ш': 26,
               'щ': 27,
               'ъ': 28,
               'ы': 29,
               'ь': 30,
               'э': 31,
               'ю': 32,
               'я': 33,
               ' ': 34}

inv_mapping = dict(zip(vocabulary.values(), vocabulary.keys()))


Считать файл с транскрипцией

In [8]:
russian = pd.read_csv('/content/gdrive/My Drive/ru_data/transcript.txt', delimiter='|')

In [9]:
del russian['trans.1']

In [10]:
del russian['numbers']

In [11]:
russian

,name,trans
0,icemarch/icemarch_0001.wav,я предпочитаю быть гильотинированным нежели ги...
1,icemarch/icemarch_0002.wav,книгу посвящаю горячо любимой матери часть пер...
2,icemarch/icemarch_0003.wav,с фронта была осень тысяча девятьсот семнадцат...
3,icemarch/icemarch_0004.wav,голубые морозные душистые бессарабские дни жел...
4,icemarch/icemarch_0005.wav,высокое золотое негреющее солнце красивый наро...
...,...,...
1911,icemarch/icemarch_1912.wav,закатное солнце бросает в воду последние лучи ...
1912,icemarch/icemarch_191.wav,будто все к чемуто прислушивается булькнули бр...
1913,icemarch/icemarch_1914.wav,прижимаясь к темнозеленому ивовому берегу толь...
1914,icemarch/icemarch_1915.wav,вместо пушек заквакали лягушки вместо пулемет...


In [12]:
maximum = 0 
b = '/content/gdrive/My Drive/ru_data/icemarch'
X = []

In [13]:
X

[]

Создание списка с числовой характеристикой для каждогго файла

In [14]:
def wav2mfcc(file_path, n_mfcc=20, max_len=1062):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=n_mfcc)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
        
    return mfcc


In [15]:
l_sound = wav2mfcc(b+'/'+soundlist[0], 20, 1062)
l_sound = l_sound[np.newaxis, :, :]
X = l_sound

In [16]:
X.shape

(1, 20, 1062)

In [17]:
for file in soundlist[1:]:
    a = str(file)
    l_sound = wav2mfcc(b+'/'+a, 20, 1062)
    l_sound = l_sound[np.newaxis, :, :]
    X = np.vstack((X, l_sound))
    

In [18]:
X.shape

(1916, 20, 1062)

In [19]:
type(X)

numpy.ndarray

Создание массив из транскрипций в таком же порядке что и численные значения

In [20]:
Y = []
for i in range(0,1916):
    Y.append(russian.loc[i, 'trans'])

In [21]:
Y

['я предпочитаю быть гильотинированным нежели гильотинирующим  дантон',
 'книгу посвящаю горячо любимой матери часть первая с фронта  до ростова',
 'с фронта была осень тысяча девятьсот семнадцатого года мы стояли в бессарабии',
 'голубые морозные душистые бессарабские дни желтокраснозеленые деревья',
 'высокое золотое негреющее солнце красивый народ в кожаных с рисунками безрукавках',
 'белые хаты внутри увешанные самоткаными коврами богатых ярких тонов я любил бессарабию',
 'по утрам полуодетый выбегаешь в сливовый сад умываешься ледяной водой пахнущей какойто особенной свежестью',
 'вбираешь грудью морозный аромат слегка заиндевевшего утра и вспоминаешь гдето читанное',
 'каждое утро душа моя у порога своего дома ты встречаешь весь мир и там же вспоминается',
 'около старенькой церкви митинги толп вооруженных людей в серых шинелях злобные речи почти без смысла',
 'знамена с надписями мир без аннексий и контрибуций долой войну',
 'смерть буржуазии',
 'речи полные злобы и ожесточения 

In [22]:
max = 0
for i in range(0, 1916):
  k = len(Y[i])
  if k>max:
    max = k
print(max)
  


297


In [23]:
len(Y)

1916

In [24]:
tr = np.array([])
for elem in Y[0]:
  number = vocabulary[elem]
  tr = np.append(tr, number)
lenght = len(Y[0])
tr = np.append(tr, [0]*(max-lenght))
tr = tr[np.newaxis, :]
completed_Y = tr

In [25]:
tr

array([[33., 34., 17., 18.,  6.,  5., 17., 16., 25., 10., 20.,  1., 32.,
        34.,  2., 29., 20., 30., 34.,  4., 10., 13., 30., 16., 20., 10.,
        15., 10., 18., 16.,  3.,  1., 15., 15., 29., 14., 34., 15.,  6.,
         8.,  6., 13., 10., 34.,  4., 10., 13., 30., 16., 20., 10., 15.,
        10., 18., 21., 32., 27., 10., 14., 34., 34.,  5.,  1., 15., 20.,
        16., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [26]:
tr.shape

(1, 297)

In [27]:
completed_Y

array([[33., 34., 17., 18.,  6.,  5., 17., 16., 25., 10., 20.,  1., 32.,
        34.,  2., 29., 20., 30., 34.,  4., 10., 13., 30., 16., 20., 10.,
        15., 10., 18., 16.,  3.,  1., 15., 15., 29., 14., 34., 15.,  6.,
         8.,  6., 13., 10., 34.,  4., 10., 13., 30., 16., 20., 10., 15.,
        10., 18., 21., 32., 27., 10., 14., 34., 34.,  5.,  1., 15., 20.,
        16., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [28]:
completed_Y.shape

(1, 297)

In [29]:
 for i in range(1,1916):
    tr = np.array([])
    for elem in Y[i]:
      number = vocabulary[elem]
      tr = np.append(tr, number)
    lenght = len(Y[i])
    tr = np.append(tr, [0]*(max-lenght))
    tr = tr[np.newaxis, :]
    completed_Y = np.vstack((completed_Y, tr))
      
    
print(completed_Y)
       
   
    
   



[[33. 34. 17. ...  0.  0.  0.]
 [12. 15. 10. ...  0.  0.  0.]
 [19. 34. 22. ...  0.  0.  0.]
 ...
 [17. 18. 10. ...  0.  0.  0.]
 [ 3. 14.  6. ...  0.  0.  0.]
 [16. 20. 34. ...  0.  0.  0.]]


In [30]:
completed_Y.shape

(1916, 297)

In [31]:
X_train, X_test, completed_Y_train, completed_Y_test = train_test_split(X, completed_Y, test_size=0.1, random_state=42)

In [32]:
print(X_train.shape)
print(X_test.shape)
print(completed_Y_train.shape)
print(completed_Y_test.shape)

(1724, 20, 1062)
(192, 20, 1062)
(1724, 297)
(192, 297)


In [47]:
model = Sequential()
model.add(LSTM(units=1062,  batch_input_shape = (1,20,1062), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=1024,return_sequences=True, activation= 'relu'))
model.add(Dropout(0.2))
model.add(LSTM(units=512, activation= 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units=297, activation= 'relu'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (1, 20, 1062)             9027000   
_________________________________________________________________
dropout_18 (Dropout)         (1, 20, 1062)             0         
_________________________________________________________________
lstm_19 (LSTM)               (1, 20, 1024)             8548352   
_________________________________________________________________
dropout_19 (Dropout)         (1, 20, 1024)             0         
_________________________________________________________________
lstm_20 (LSTM)               (1, 512)                  3147776   
_________________________________________________________________
dropout_20 (Dropout)         (1, 512)                  0         
_________________________________________________________________
dense_6 (Dense)              (1, 297)                 

In [ ]:
model.fit(X_train, completed_Y_train, epochs=10, batch_size= 1724, validation_batch_size=192, validation_data=(X_test, completed_Y_test))

In [50]:
test = wav2mfcc(b+'/'+soundlist[0], 1, 1062)
test = test[np.newaxis, :,:]

In [51]:
model.predict(test)

array([[0.19097313, 0.11694564, 0.12770356, 0.14756751, 0.13873138,
        0.14608385, 0.13601106, 0.        , 0.10154705, 0.11037641,
        0.13054943, 0.15412618, 0.12790145, 0.13493891, 0.19392672,
        0.11485805, 0.1287998 , 0.12745285, 0.17703801, 0.15624249,
        0.2107229 , 0.10830312, 0.16898221, 0.08656514, 0.14481091,
        0.12264976, 0.15707083, 0.1455047 , 0.16423221, 0.05691147,
        0.15309821, 0.12346616, 0.1831852 , 0.18690133, 0.13688992,
        0.21421517, 0.10710988, 0.12541455, 0.21498455, 0.03034986,
        0.20200306, 0.11734976, 0.        , 0.14470744, 0.1625617 ,
        0.11550974, 0.1065028 , 0.08931633, 0.20159599, 0.07986582,
        0.11190031, 0.17847034, 0.147286  , 0.14110103, 0.13822788,
        0.1473004 , 0.106938  , 0.07980013, 0.13025706, 0.1279949 ,
        0.15302105, 0.15126266, 0.10379439, 0.13476446, 0.        ,
        0.12281401, 0.07958856, 0.14293808, 0.11508873, 0.08058199,
        0.21835136, 0.1100178 , 0.13541776, 0.03

In [ ]:
for layer in model.layers:
    print(layer.output_shape)